In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import Input, LocallyConnected1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 400
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 400
N_FEATURES = 12

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 60, 32

In [12]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 20.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [13]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
# import os
path='/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/'
# path='/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nbov_olsson/sub_wise_process_TT/'

In [15]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],400,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [16]:
n_length =  400
n_depth=12
n_channel=12
n_outputs=49

In [17]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,12)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model   

In [18]:
if __name__ == "__main__":
    model = generate_model()

(None, 400, 12)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
ip_shape (None, 400)
ip_shape (None, 400, 1)
conv1_o (None, 400, 64)
(None, 400, 12, 64)
<class 'tensorflow.python.framework.ops.Tensor'>
(None, 307200)
(None, 49)
inputs_shape (None, 400, 12)


In [19]:
for i in range(1,41):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
#     feature standardization
    scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])
    df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])
    df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])
    
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_length =  400
    n_depth=12
    n_channel=12
    x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5'
#     model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)
    test_mean=statistics.mean(test_acc)
    print('test_mean for %d subjects:',i)
    print(test_mean)

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/S30_tr.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 1960000

/media/naveen/nav/mat_codes/nina_DB2_codes/prep_nl/sub_wise_process_TT/S30_tt.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 980000

x_train shape:  (4899, 400, 12)
4899 training samples
y_train shape:  (4899,)
num_time_periods 400
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0]
x_train shape: (400, 12)
input_shape: (400, 12)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train shape:  (4899, 49)
x_test shape:  (2449, 400, 12

Epoch 1/60
154/154 [==============================] - ETA: 0s - loss: 5.4007 - accuracy: 0.1272
Epoch 00001: val_accuracy improved from -inf to 0.27807, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 152s 985ms/step - loss: 5.4007 - accuracy: 0.1272 - val_loss: 4.0496 - val_accuracy: 0.2781 - lr: 0.0010
Epoch 2/60
154/154 [==============================] - ETA: 0s - loss: 3.6237 - accuracy: 0.2868
Epoch 00002: val_accuracy improved from 0.27807 to 0.37036, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 61s 396ms/step - loss: 3.6237 - accuracy: 0.2868 - val_loss: 3.0361 - val_accuracy: 0.3704 - lr: 0.0010
Epoch 3/60
154/154 [==============================] - ETA: 0s - loss: 2.8794 - accuracy: 0.3733
Epoch 00003: val_accuracy improved from 0.37036 to 0.41731, saving model to /media/navee

Epoch 50/60
154/154 [==============================] - ETA: 0s - loss: 1.0750 - accuracy: 0.7710
Epoch 00050: val_accuracy did not improve from 0.54594
154/154 [==============================] - 53s 346ms/step - loss: 1.0750 - accuracy: 0.7710 - val_loss: 2.1302 - val_accuracy: 0.5051 - lr: 1.0000e-05
Epoch 51/60
154/154 [==============================] - ETA: 0s - loss: 1.0806 - accuracy: 0.7655
Epoch 00051: val_accuracy did not improve from 0.54594
154/154 [==============================] - 56s 361ms/step - loss: 1.0806 - accuracy: 0.7655 - val_loss: 1.9776 - val_accuracy: 0.5149 - lr: 1.0000e-05
Epoch 52/60
154/154 [==============================] - ETA: 0s - loss: 1.0888 - accuracy: 0.7624
Epoch 00052: val_accuracy did not improve from 0.54594
154/154 [==============================] - 53s 347ms/step - loss: 1.0888 - accuracy: 0.7624 - val_loss: 1.9568 - val_accuracy: 0.5247 - lr: 1.0000e-05
Epoch 53/60
154/154 [==============================] - ETA: 0s - loss: 1.0802 - accuracy: 0

Epoch 1/60
154/154 [==============================] - ETA: 0s - loss: 3.9672 - accuracy: 0.1976
Epoch 00001: val_accuracy improved from -inf to 0.34218, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 125s 811ms/step - loss: 3.9672 - accuracy: 0.1976 - val_loss: 2.7501 - val_accuracy: 0.3422 - lr: 0.0010
Epoch 2/60
154/154 [==============================] - ETA: 0s - loss: 2.6853 - accuracy: 0.3572
Epoch 00002: val_accuracy improved from 0.34218 to 0.42915, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 61s 396ms/step - loss: 2.6853 - accuracy: 0.3572 - val_loss: 2.3538 - val_accuracy: 0.4292 - lr: 0.0010
Epoch 3/60
154/154 [==============================] - ETA: 0s - loss: 2.4132 - accuracy: 0.4364
Epoch 00003: val_accuracy improved from 0.42915 to 0.44426, saving model to /media/navee

Epoch 25/60
154/154 [==============================] - ETA: 0s - loss: 1.1103 - accuracy: 0.8198
Epoch 00025: val_accuracy did not improve from 0.54920
154/154 [==============================] - 52s 336ms/step - loss: 1.1103 - accuracy: 0.8198 - val_loss: 2.1430 - val_accuracy: 0.5382 - lr: 1.0000e-04
Epoch 26/60
154/154 [==============================] - ETA: 0s - loss: 1.0503 - accuracy: 0.8373
Epoch 00026: val_accuracy did not improve from 0.54920
154/154 [==============================] - 52s 337ms/step - loss: 1.0503 - accuracy: 0.8373 - val_loss: 2.1301 - val_accuracy: 0.5439 - lr: 1.0000e-04
Epoch 27/60
154/154 [==============================] - ETA: 0s - loss: 1.0290 - accuracy: 0.8385
Epoch 00027: val_accuracy did not improve from 0.54920
154/154 [==============================] - 52s 336ms/step - loss: 1.0290 - accuracy: 0.8385 - val_loss: 2.2510 - val_accuracy: 0.4982 - lr: 1.0000e-04
Epoch 28/60
154/154 [==============================] - ETA: 0s - loss: 1.0290 - accuracy: 0

...

Epoch 1/60
154/154 [==============================] - ETA: 0s - loss: 4.9421 - accuracy: 0.1259
Epoch 00001: val_accuracy improved from -inf to 0.24051, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 130s 845ms/step - loss: 4.9421 - accuracy: 0.1259 - val_loss: 3.3557 - val_accuracy: 0.2405 - lr: 0.0010
Epoch 2/60
154/154 [==============================] - ETA: 0s - loss: 3.2622 - accuracy: 0.2568
Epoch 00002: val_accuracy improved from 0.24051 to 0.31727, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 59s 384ms/step - loss: 3.2622 - accuracy: 0.2568 - val_loss: 2.9894 - val_accuracy: 0.3173 - lr: 0.0010
Epoch 3/60
154/154 [==============================] - ETA: 0s - loss: 2.9792 - accuracy: 0.3548
Epoch 00003: val_accuracy improved from 0.31727 to 0.36913, saving model to /media/navee

Epoch 26/60
154/154 [==============================] - ETA: 0s - loss: 1.4225 - accuracy: 0.8134
Epoch 00026: val_accuracy did not improve from 0.43732
154/154 [==============================] - 49s 321ms/step - loss: 1.4225 - accuracy: 0.8134 - val_loss: 3.1570 - val_accuracy: 0.4230 - lr: 1.0000e-04
Epoch 27/60
154/154 [==============================] - ETA: 0s - loss: 1.4167 - accuracy: 0.8034
Epoch 00027: val_accuracy did not improve from 0.43732
154/154 [==============================] - 50s 322ms/step - loss: 1.4167 - accuracy: 0.8034 - val_loss: 3.0792 - val_accuracy: 0.4287 - lr: 1.0000e-04
Epoch 28/60
154/154 [==============================] - ETA: 0s - loss: 1.3754 - accuracy: 0.8122
Epoch 00028: val_accuracy did not improve from 0.43732
154/154 [==============================] - 49s 321ms/step - loss: 1.3754 - accuracy: 0.8122 - val_loss: 3.1423 - val_accuracy: 0.4247 - lr: 1.0000e-04
Epoch 29/60
154/154 [==============================] - ETA: 0s - loss: 1.3325 - accuracy: 0

154/154 [==============================] - ETA: 0s - loss: 1.1290 - accuracy: 0.8483
Epoch 00053: val_accuracy did not improve from 0.43732
154/154 [==============================] - 50s 321ms/step - loss: 1.1290 - accuracy: 0.8483 - val_loss: 3.1121 - val_accuracy: 0.4177 - lr: 1.0000e-05
Epoch 54/60
154/154 [==============================] - ETA: 0s - loss: 1.1440 - accuracy: 0.8526
Epoch 00054: val_accuracy did not improve from 0.43732
154/154 [==============================] - 49s 320ms/step - loss: 1.1440 - accuracy: 0.8526 - val_loss: 3.1228 - val_accuracy: 0.4173 - lr: 1.0000e-05
Epoch 55/60
154/154 [==============================] - ETA: 0s - loss: 1.0898 - accuracy: 0.8673
Epoch 00055: val_accuracy did not improve from 0.43732
154/154 [==============================] - 50s 322ms/step - loss: 1.0898 - accuracy: 0.8673 - val_loss: 3.0816 - val_accuracy: 0.4120 - lr: 1.0000e-05
Epoch 56/60
154/154 [==============================] - ETA: 0s - loss: 1.1343 - accuracy: 0.8516
Epoch 

Epoch 1/60
154/154 [==============================] - ETA: 0s - loss: 5.4557 - accuracy: 0.1282
Epoch 00001: val_accuracy improved from -inf to 0.28338, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 127s 822ms/step - loss: 5.4557 - accuracy: 0.1282 - val_loss: 3.3481 - val_accuracy: 0.2834 - lr: 0.0010
Epoch 2/60
154/154 [==============================] - ETA: 0s - loss: 3.2230 - accuracy: 0.3227
Epoch 00002: val_accuracy improved from 0.28338 to 0.42752, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/wei_DB2_20X10/checkpoint.hdf5
154/154 [==============================] - 57s 373ms/step - loss: 3.2230 - accuracy: 0.3227 - val_loss: 2.7517 - val_accuracy: 0.4275 - lr: 0.0010
Epoch 3/60
154/154 [==============================] - ETA: 0s - loss: 2.7064 - accuracy: 0.4252
Epoch 00003: val_accuracy improved from 0.42752 to 0.47570, saving model to /media/navee

Epoch 50/60
154/154 [==============================] - ETA: 0s - loss: 0.8062 - accuracy: 0.9222
Epoch 00050: val_accuracy did not improve from 0.55574
154/154 [==============================] - 51s 331ms/step - loss: 0.8062 - accuracy: 0.9222 - val_loss: 2.3332 - val_accuracy: 0.5419 - lr: 1.0000e-05
Epoch 51/60
154/154 [==============================] - ETA: 0s - loss: 0.8144 - accuracy: 0.9179
Epoch 00051: val_accuracy did not improve from 0.55574
154/154 [==============================] - 51s 332ms/step - loss: 0.8144 - accuracy: 0.9179 - val_loss: 2.3711 - val_accuracy: 0.5447 - lr: 1.0000e-05
Epoch 52/60
154/154 [==============================] - ETA: 0s - loss: 0.8237 - accuracy: 0.9177
Epoch 00052: val_accuracy did not improve from 0.55574
154/154 [==============================] - 51s 333ms/step - loss: 0.8237 - accuracy: 0.9177 - val_loss: 2.3518 - val_accuracy: 0.5517 - lr: 1.0000e-05
Epoch 53/60
154/154 [==============================] - ETA: 0s - loss: 0.8097 - accuracy: 0

In [20]:
statistics.mean(test_acc)

0.5225880687887018